# OpenSearch 한글 형태소 분석기 통한 키워드 검색 
>이 노트북은,
> - SageMaker Studio* **`Data Science 3.0`** kernel 및 ml.t3.medium 인스턴스에서 테스트 되었습니다.
> - SageMaker Notebook **`conda_python3`** 에서 테스트 되었습니다.


여기서는 OpenSearch 가 설치된 것을 가정하고, 한글 형태소 분석기의 사용하는 법을 알려 드립니다.

---

### [중요]
- 이 노트북은 Bedrock Titan Embedding Model 을 기본으로 사용합니다. KoSIMCSERoberta 를 세이지 메이커 엔드포인트로 사용하신다면 아래의 선수 조건을 확인하세요.

#### 선수조건 (KoSIMCSERoberta 사용시)
- 임베딩 모델의 세이지 메이커 엔드포인트가 액티브 된 상태를 가정 합니다.
    - 세이지 메이커 엔드포인트에 배포하기 위해서는 아래 노트북을 실행하시고, Endpoint Name 만을 복사 하시면 됩니다.
    - [KoSIMCSERoberta Embedding Model 배포](https://github.com/gonsoomoon-ml/Kor-LLM-On-SageMaker/blob/main/1-Lab01-Deploy-LLM/4.Kor-Embedding-Model.ipynb)
    - SageMaker Endpoint 에 대해서는 공식 개발자 문서를 참조하세요 --> [Create your endpoint and deploy your model](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-deployment.html)
- 오픈 서치 서비스가 액티브 된 상태를 가정 합니다.


---
## Ref: 
- [Amazon OpenSearch Service로 검색 구현하기](https://catalog.us-east-1.prod.workshops.aws/workshops/de4e38cb-a0d9-4ffe-a777-bf00d498fa49/ko-KR/indexing/blog-reindex)
- [OpenSearch Python Client](https://opensearch.org/docs/1.3/clients/python-high-level/)
- [OpenSearch Match, Multi-Match, and Match Phrase Queries](https://opster.com/guides/opensearch/opensearch-search-apis/opensearch-match-multi-match-and-match-phrase-queries/)
- OpenSearch Query 에서 Filter, Must, Should, Not Mush 에 대한 설명 입니다.
    - [OpenSearch Boolean Queries](https://opster.com/guides/opensearch/opensearch-search-apis/opensearch-boolean-queries/#:~:text=Boolean%20queries%20are%20used%20to,as%20terms%2C%20match%20and%20query_string.)
- [OpenSearch Query Description (한글)](https://esbook.kimjmin.net/05-search)


# 1. 환경 세팅

In [1]:
import boto3
region = boto3.Session().region_name
opensearch = boto3.client('opensearch', region)

%store -r opensearch_user_id opensearch_user_password domain_name opensearch_domain_endpoint

try:
    opensearch_user_id
    opensearch_user_password
    domain_name
    opensearch_domain_endpoint
   
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] Run 00_setup notebook first or Create Your Own OpenSearch Domain")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [2]:
%load_ext autoreload
%autoreload 2

import sys, os
module_path = ".."
sys.path.append(os.path.abspath(module_path))

## Bedrock Client 생성
### 선수 지식
아래의 노트북을 먼저 실행해서, Bedrock 에 접근 가능하게 합니다.
- amazon-bedrock-workshop-webinar-kr/00_Setup/setup.ipynb

In [3]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

# print (colored("\n== FM lists ==", "green"))
# pprint (bedrock_info.get_list_fm_models())

bedrock = boto3.client(service_name='bedrock')
model_list = bedrock.list_foundation_models()
result = [(fm_list["modelName"], fm_list["modelId"]) for fm_list in model_list["modelSummaries"] if fm_list['inferenceTypesSupported'] == ['ON_DEMAND']]
pprint(result)

Create new client
  Using region: None
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-west-2.amazonaws.com)
[('Titan Text Large', 'amazon.titan-tg1-large'),
 ('Titan Text Embeddings v2', 'amazon.titan-embed-g1-text-02'),
 ('Titan Text G1 - Lite', 'amazon.titan-text-lite-v1'),
 ('Titan Text G1 - Express', 'amazon.titan-text-express-v1'),
 ('Titan Embeddings G1 - Text', 'amazon.titan-embed-text-v1'),
 ('Titan Multimodal Embeddings G1', 'amazon.titan-embed-image-v1'),
 ('Titan Image Generator G1', 'amazon.titan-image-generator-v1'),
 ('SDXL 0.8', 'stability.stable-diffusion-xl'),
 ('SDXL 0.8', 'stability.stable-diffusion-xl-v0'),
 ('SDXL 1.0', 'stability.stable-diffusion-xl-v1'),
 ('J2 Grande Instruct', 'ai21.j2-grande-instruct'),
 ('J2 Jumbo Instruct', 'ai21.j2-jumbo-instruct'),
 ('Jurassic-2 Mid', 'ai21.j2-mid'),
 ('Jurassic-2 Mid', 'ai21.j2-mid-v1'),
 ('Jurassic-2 Ultra', 'ai21.j2-ultra'),
 ('Jurassic-2 Ultra', 'ai21.j2-ultra

# 2. Titan Embedding 및 LLM 인 Claude-v2 모델 로딩

## LLM 로딩 (Claude-v2.1)

In [36]:
from langchain.llms.bedrock import Bedrock
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [37]:
# - create the Anthropic Model
llm_text = Bedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-V2-1"),
    client=boto3_bedrock,
    model_kwargs={
        "max_tokens_to_sample": 512
    },
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)
llm_text

Bedrock(client=<botocore.client.BedrockRuntime object at 0x7f987be08520>, model_id='anthropic.claude-v2:1', model_kwargs={'max_tokens_to_sample': 512}, streaming=True, callbacks=[<langchain_core.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x7f979768a560>])

## Embedding 모델 선택

In [4]:
from utils.rag import KoSimCSERobertaContentHandler, SagemakerEndpointEmbeddingsJumpStart

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [5]:
def get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name=None):
    if is_bedrock_embeddings:

        # We will be using the Titan Embeddings Model to generate our Embeddings.
        from langchain.embeddings import BedrockEmbeddings

        llm_emb = BedrockEmbeddings(
            client=boto3_bedrock,
            # model_id="cohere.embed-multilingual-v3"
            model_id="amazon.titan-embed-g1-text-02"
        )
        print("Bedrock Embeddings Model Loaded")
        
    elif is_KoSimCSERobert:
        LLMEmbHandler = KoSimCSERobertaContentHandler()
        endpoint_name_emb = endpont_name
        llm_emb = SagemakerEndpointEmbeddingsJumpStart(
            endpoint_name=endpoint_name_emb,
            region_name=aws_region,
            content_handler=LLMEmbHandler,
        )        
        print("KoSimCSERobert Embeddings Model Loaded")
    else:
        llm_emb = None
        print("No Embedding Model Selected")
    
    return llm_emb

#### [중요] is_KoSimCSERobert == True 경우,  endpoint_name 을 꼭 넣어 주세요.

In [6]:
is_bedrock_embeddings = True
is_KoSimCSERobert = False

aws_region = os.environ.get("AWS_DEFAULT_REGION", None)

##############################
# Parameters for is_KoSimCSERobert
##############################
if is_KoSimCSERobert: endpont_name = "<endpoint-name>"
else: endpont_name = None
##############################

llm_emb = get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name)    

Bedrock Embeddings Model Loaded


-------------------

# 3. OpenSearch 벡터 Indexer 생성
### 선수 조건
- 랭체인 오프서처 참고 자료
    - [Langchain Opensearch](https://python.langchain.com/docs/integrations/vectorstores/opensearch)

## 오픈 서치 인덱스 유무에 따라 삭제
오픈 서치에 해당 인덱스가 존재하면, 삭제 합니다. 

In [7]:
from opensearchpy import OpenSearch, RequestsHttpConnection
http_auth = (opensearch_user_id, opensearch_user_password)
os_client = OpenSearch(
            hosts=[
                {'host': opensearch_domain_endpoint.replace("https://", ""),
                 'port': 443
                }
            ],
            http_auth=http_auth, # Master username, Master password,
            use_ssl=True,
            verify_certs=True,
            connection_class=RequestsHttpConnection
        )

## 형태소 분석기 사용하기
- 영어권의 문자들과 다르게 한글, 일본어, 중국어 등은 단순한 공백만으로는 좋은 검색 결과를 얻기 힘듭니다.
- 출시하고라는 단어가 들어간 문서를 출시하고라는 정확히 같은 단어만으로 검색할 수 있다면 답답하겠죠?
- 출시하고라는 단어를 출시, 출시하고 등 다양하게 검색하기 위해서는 형태소 분석기가 필요합니다.
- OpenSearch 에서는 2개의 한국어 analyer를 제공하고 있습니다.
    - 은전한잎 (seunjeon_tokenizer)
        - https://catalog.us-east-1.prod.workshops.aws/workshops/de4e38cb-a0d9-4ffe-a777-bf00d498fa49/ko-KR/indexing/stemming#
    - Nori (nori_tokenizer)
        - 설명: https://esbook.kimjmin.net/06-text-analysis/6.7-stemming/6.7.2-nori
    - Sample 코드에서는 "Nori"를 기반으로 진행합니다.

### [TIP]
- **token filter**
    - Amazon OpenSearch Service에서는 나만의 필터를 토크나이저와 함께 구성해서 사용할 수 있습니다. 
    - nori_tokenizer를 사용하되, 원하는 캐릭터 필터와 토큰 필터 조합을 구성할 수 있습니다.
    - 문장이 입력되면 캐릭터 필터(char_filter), 토크나이저(tokenizer), 토큰 필터(filter) 순으로 동작하게 됩니다.
    - 입력문장이 "<b>Start</b> 이천이십삼년 韓國" 이것일 경우, 
    - 아래 샘플 코드의 옵션으로 사용한다면 <b>Start</b>의 html 코드가 html_strip에 의해 처리되고, nori_tokenizer로 토큰화됩니다.
    - 그 후에 nori_number는 이천이십삼을 2023으로, nori_readingform은 韓國을 한국으로, lowercase는 Start를 start로 처리합니다.
    - **토큰필터 참고자료**
        - https://esbook.kimjmin.net/06-text-analysis/6.6-token-filter
        - https://opensearch.org/docs/latest/analyzers/token-filters/index/
    
- **discard_punctuation**
    - true(기본값), false가 있으며 문장부호 또는 구두점을 어떻게 다룰지에 대한 설정을 할 수 있습니다.
- **decompound_mode**
    - none, discard(기본값), mixed가 있으며 복합명사를 어떻게 다룰지에 대한 설정을 할 수 있습니다.
    - none: 복합명사를 분리하지 않고 하나의 토큰으로 저장합니다.
    - discard: 복합명사를 분리하여 토큰으로 저장합니다.
    - mixed: 복합명사를 분리하지 않은 토큰과 분리한 토큰을 모두 저장합니다.
- **신조어, 업무용어, 상표**
     - 노리 토크나이저는 [mecab-ko-dic](https://bitbucket.org/eunjeon/mecab-ko-dic/src/master/) 을 사용하고 있지만 때로는 신조어, 업무 용어, 상표 등을 위한 사용자 사전이 필요할 수 있습니다.
     - user_dictionary_rules를 이용해서 사용자 사전을 만들 수 있고, 아래에서 사용자 사전을 적용했을 때와 적용하지 않았을 때를 비교해 볼 수 있습니다.
- **동의어, 불용어**
    - Amazon OpenSearch Service에서는 Package를 이용하면 사용자사전, 동의어, 불용어를 관리할 수 있습니다. 자세한 사항은 [이곳](https://docs.aws.amazon.com/ko_kr/opensearch-service/latest/developerguide/custom-packages.html)을 참조해 주십시오. 
    - S3에 사용자 사전을 업로드 후, Package에 등록합니다. 그리고 등록된 텍스트 사전을 원하는 OpenSearch 도메인에 연결합니다.
    - 아래와 같이 user_dictionary에 analyzers/<Package ID> 를 이용해서 적용할 수 있습니다. 이 Package ID는 OpenSearch 콘솔 화면에서 업로드한 사전의 상세 페이지에서 확인할 수 있습니다.


## 인덱스 생성

In [8]:
index_name = "genai-demo-index-v1"
exists = os_client.indices.exists(index_name)

if exists:
    os_client.indices.delete(index=index_name)
    print("Index is deleted")
else:
    print("Index does not exist")

Index is deleted


In [9]:
### model에 따라 dimension 사이즈 변경 필요 (Titan : 1536, Cohere : 1024)
import json

with open('index_body.json', 'r') as f:
    index_body = json.load(f)

index_body

{'mappings': {'properties': {'metadata': {'properties': {'source': {'type': 'keyword'},
     'timestamp': {'type': 'float'},
     'type': {'type': 'keyword'}}},
   'text': {'analyzer': 'custom_analyzer',
    'fields': {'keyword': {'ignore_above': 256, 'type': 'keyword'}},
    'search_analyzer': 'custom_analyzer',
    'type': 'text'},
   'vector_field': {'dimension': 1536,
    'method': {'engine': 'faiss',
     'name': 'hnsw',
     'parameters': {'ef_construction': 512, 'm': 16},
     'space_type': 'l2'},
    'type': 'knn_vector'}}},
 'settings': {'analysis': {'analyzer': {'custom_analyzer': {'char_filter': ['html_strip'],
     'filter': ['nori_number', 'nori_readingform', 'lowercase'],
     'tokenizer': 'nori',
     'type': 'custom'}},
   'tokenizer': {'nori': {'decompound_mode': 'mixed',
     'discard_punctuation': 'true',
     'type': 'nori_tokenizer'}}},
  'index': {'knn': 'true',
   'knn.algo_param': {'ef_search': '512'},
   'number_of_replicas': '2',
   'number_of_shards': '5'}}}

In [10]:
os_client.indices.create(index_name, body=index_body)

{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'genai-demo-index-v1'}

In [11]:
%%time
from langchain.vectorstores import OpenSearchVectorSearch

vector_db = OpenSearchVectorSearch(
    index_name=index_name,
    opensearch_url=opensearch_domain_endpoint,
    embedding_function=llm_emb,
    http_auth=http_auth, # http_auth
)

CPU times: user 258 µs, sys: 40 µs, total: 298 µs
Wall time: 305 µs


# 4. 데이터 준비


##  한화생명 보험 약관 데이터 세트로 구현

In [12]:
import time
from langchain.schema import Document
from llmsherpa.readers import LayoutPDFReader

In [13]:
import glob
pdf_list = glob.glob('./data/rag_data_kr_pdf/*')
pdf_list

['./data/rag_data_kr_pdf/한화생명 간편가입 H튼튼 보장보험_20240101~.pdf',
 './data/rag_data_kr_pdf/한화생명 간편가입 H튼튼 보장보험_20230601~.pdf',
 './data/rag_data_kr_pdf/한화생명 The스마트한 일시납종신보험_20230916~.pdf',
 './data/rag_data_kr_pdf/한화생명 간편가입 웰다잉 종신보험_20240101~.pdf',
 './data/rag_data_kr_pdf/한화생명 The스마트한 일시납종신보험_20240101~.pdf',
 './data/rag_data_kr_pdf/한화생명 The실속있는 간병치매보험_20231209~_1.pdf',
 './data/rag_data_kr_pdf/한화생명 간편가입 웰다잉 종신보험_20230901~.pdf',
 './data/rag_data_kr_pdf/한화생명 간편가입 웰다잉 종신보험_20230916~.pdf',
 './data/rag_data_kr_pdf/한화생명 The실속있는 간병치매보험_20240101~.pdf']

### 1. PDF 처리 방법 (LLM Sherpa)
 - https://github.com/nlmatics/llmsherpa

In [14]:
# llmsherpa_api_url = "https://readers.llmsherpa.com/api/document/developer/parseDocument?renderFormat=all"
# pdf_reader = LayoutPDFReader(llmsherpa_api_url)
# # doc = pdf_reader.read_pdf("./data/pdf/한화생명 The스마트한 일시납종신보험_20240101~.pdf")

In [15]:
# docs = []
# for pdf_file in pdf_list:
#     doc = pdf_reader.read_pdf(pdf_file)
#     source_name = pdf_file.split('/')[-1]
#     type_name = source_name.split(' ')[-1].replace('.pdf', '')
#     for chunk_info in doc.chunks():
#         sentences = " ".join(chunk_info.sentences)
#         if len(sentences) >= 20:
#             chunk = Document(
#                 page_content=sentences,
#                 metadata={
#                     "source" : source_name,
#                     "type": type_name,
#                     "timestamp": time.time()
#                 }
#             )
#             docs.append(chunk)

### 2. PDF 처리 방법 (pypdf)
 - https://pypi.org/project/pypdf/

### 3. PDF 처리 방법 (pdfplumber)
 - https://github.com/jsvine/pdfplumber

In [16]:
from multiprocessing.pool import ThreadPool
from multiprocessing import  Manager

import pdfplumber
import glob

In [17]:
def read_pdf(param):
    vector_db = param[0]
    current_pdf_file = param[1]
    print(f"current_pdf_file : {current_pdf_file}")
    docs = []
    source_name = current_pdf_file.split('/')[-1]
    type_name = source_name.split(' ')[-1].replace('.pdf', '')
    with pdfplumber.open(current_pdf_file) as my_pdf:
        pages = my_pdf.pages
        for page in pages:
            sentences = page.extract_text()
            if len(sentences) >= 20:
                chunk = Document(
                    page_content=sentences,
                    metadata={
                        "source" : source_name,
                        "type": type_name,
                        "timestamp": time.time()
                    }
                )
                # print(f"chunk : {chunk}")
                docs.append(chunk)
    vector_db.add_documents(docs)

In [18]:
manager = Manager()
result_dict = manager.dict()

param = [(vector_db, current_pdf_file)for current_pdf_file in glob.glob("./data/rag_data_kr_pdf/*.pdf")]

with ThreadPool(processes=4) as pool:
    pool.map(read_pdf, param)
    pool.close()
    pool.join()

current_pdf_file : ./data/rag_data_kr_pdf/한화생명 간편가입 H튼튼 보장보험_20240101~.pdf
current_pdf_file : ./data/rag_data_kr_pdf/한화생명 간편가입 H튼튼 보장보험_20230601~.pdf
current_pdf_file : ./data/rag_data_kr_pdf/한화생명 The스마트한 일시납종신보험_20230916~.pdf
current_pdf_file : ./data/rag_data_kr_pdf/한화생명 간편가입 웰다잉 종신보험_20240101~.pdf
current_pdf_file : ./data/rag_data_kr_pdf/한화생명 The스마트한 일시납종신보험_20240101~.pdf
current_pdf_file : ./data/rag_data_kr_pdf/한화생명 The실속있는 간병치매보험_20231209~_1.pdf
current_pdf_file : ./data/rag_data_kr_pdf/한화생명 간편가입 웰다잉 종신보험_20230901~.pdf
current_pdf_file : ./data/rag_data_kr_pdf/한화생명 간편가입 웰다잉 종신보험_20230916~.pdf
current_pdf_file : ./data/rag_data_kr_pdf/한화생명 The실속있는 간병치매보험_20240101~.pdf


In [19]:
# import pdfplumber
# import glob
# from tqdm.auto import tqdm

# docs = []
# for current_pdf_file in glob.glob("./data/rag_data_kr_pdf/*.pdf"):
#     source_name = current_pdf_file.split('/')[-1]
#     type_name = source_name.split(' ')[-1].replace('.pdf', '')
#     with pdfplumber.open(current_pdf_file) as my_pdf:
#         pages = my_pdf.pages
#         for page in pages:
#             sentences = page.extract_text()
#             if len(sentences) >= 20:
#                 chunk = Document(
#                     page_content=sentences,
#                     metadata={
#                         "source" : source_name,
#                         "type": type_name,
#                         "timestamp": time.time()
#                     }
#                 )
#                 docs.append(chunk)
#     break

## 인덱스 확인

In [20]:
index_info = os_client.indices.get(index=index_name)
pprint(index_info)

{'genai-demo-index-v1': {'aliases': {},
                         'mappings': {'properties': {'metadata': {'properties': {'source': {'type': 'keyword'},
                                                                                 'timestamp': {'type': 'float'},
                                                                                 'type': {'type': 'keyword'}}},
                                                     'text': {'analyzer': 'custom_analyzer',
                                                              'fields': {'keyword': {'ignore_above': 256,
                                                                                     'type': 'keyword'}},
                                                              'type': 'text'},
                                                     'vector_field': {'dimension': 1536,
                                                                      'method': {'engine': 'faiss',
                                                  

# 4. 키워드 검색

### 'Text" 에 "보험" 단어를 검색합니다.

In [21]:
from opensearch_dsl import Search

In [22]:
def result_to_dataframe(response):
    import pandas as pd

    pd.set_option('display.max_columns', 150)
    pd.set_option('display.max_colwidth', 200)
    result = []
    for res in response['hits']['hits']:
        # print(res.keys())
        result.append([res['_index'], round(res['_score'], 4), res['_source']['metadata']['type'], res['_source']['text']])
    return pd.DataFrame(result, columns=['index_name', 'score', 'type', 'text'])

In [23]:
def query_lexical(query, filter=[], k=5):
    QUERY_TEMPLATE = {
        "size": k,
        "query": {
            "bool": {
                "must": [
                    {
                        "match": {
                            "text": {
                                "query": query,
                                "operator":  "or"
                            }
                        }
                    }
                ],
                "filter": filter
            }
        }
    }
    if len(filter) > 0:
        QUERY_TEMPLATE["query"]["bool"]["filter"].extend(filter)
    return QUERY_TEMPLATE

In [25]:
query = "계약 전 의무 사항은 무엇인가요"

response = os_client.search(
    body=query_lexical(query),
    index=index_name
)
result_to_dataframe(response)

,index_name,score,type,text
0,genai-demo-index-v1,30.9541,종신보험_20230901~,4. 상황별 목차 안내\n계약의 계약에서 쓰이는\n제2조(용어의 정의)\n체결 용어를 알고 싶어요\n제14조(계약 전 알릴 의무)\n계약 전 알릴 의무는 무엇인가요\n제15조(계약 전 알릴 의무 위반의 효과)\n보장이 언제 시작되나요 제24조(제1회 보험료 및 회사의 보장개시)\n청약을 철회하고 싶어요 제18조(청약의 철회)\n계약을 취소 할 수 있...
1,genai-demo-index-v1,30.9541,종신보험_20230916~,4. 상황별 목차 안내\n계약의 계약에서 쓰이는\n제2조(용어의 정의)\n체결 용어를 알고 싶어요\n제14조(계약 전 알릴 의무)\n계약 전 알릴 의무는 무엇인가요\n제15조(계약 전 알릴 의무 위반의 효과)\n보장이 언제 시작되나요 제24조(제1회 보험료 및 회사의 보장개시)\n청약을 철회하고 싶어요 제18조(청약의 철회)\n계약을 취소 할 수 있...
2,genai-demo-index-v1,29.9808,일시납종신보험_20230916~,(2) 해약환급금 미보증형\n계약의 계약에서 쓰이는\n제2조(용어의 정의)\n체결 용어를 알고 싶어요\n제14조(계약 전 알릴 의무)\n계약 전 알릴 의무는 무엇인가요\n제15조(계약 전 알릴 의무 위반의 효과)\n제25조(제1회 보험료 및 회사의\n보장이 언제 시작되나요\n보장개시)\n청약을 철회하고 싶어요 제18조(청약의 철회)\n계약을 취소 할...
3,genai-demo-index-v1,29.9808,보장보험_20240101~,4. 상황별 목차 안내\n계약의 계약에서 쓰이는\n제2조(용어의정의)\n체결 용어를 알고 싶어요\n제16조(계약 전 알릴 의무)\n계약 전 알릴 의무는 무엇인가요\n제17조(계약 전 알릴 의무 위반의 효과)\n보장이 언제 시작되나요 제26조(제1회 보험료 및 회사의 보장개시)\n청약을 철회하고 싶어요 제20조(청약의 철회)\n계약을 취소 할 수 있나...
4,genai-demo-index-v1,29.9808,간병치매보험_20231209~_1,4. 상황별 목차 안내\n계약의 계약에서 쓰이는\n제2조(용어의 정의)\n체결 용어를 알고 싶어요\n제14조(계약 전 알릴 의무)\n계약 전 알릴 의무는 무엇인가요\n제15조(계약 전 알릴 의무 위반의 효과)\n보장이 언제 시작되나요 제24조(제1회 보험료 및 회사의 보장개시)\n청약을 철회하고 싶어요 제18조(청약의 철회)\n계약을 취소 할 수 있...


## 5. Filter 활용
- document내 metadata를 활용하여 search space를 줄일 수 있다.
- 특히 filter의 경우 search 전에 수행되기 때문에, 검색 속도 향상을 기대할 수 있다
- syntax
    - filter=[{"term"**[고정]**: {"metadata.source"**[메타데이터 이름, 혹은 메타데이터 아니여도 상관없음]**: "신한은행"**[조건명]**}},]
    - list 형식으로 복수개 filter 설정 가능

In [26]:
query = "계약 전 의무 사항은 무엇인가요"

filter = [
    {"term": {"metadata.source": "한화생명 The스마트한 일시납종신보험_20240101~.pdf"}},
    {"term": {"metadata.type": "일시납종신보험_20240101~"}},
]


response = os_client.search(
    body=query_lexical(query, filter),
    index=index_name
)
result_to_dataframe(response)

,index_name,score,type,text
0,genai-demo-index-v1,29.8904,일시납종신보험_20240101~,(2) 해약환급금 미보증형\n계약의 계약에서 쓰이는\n제2조(용어의 정의)\n체결 용어를 알고 싶어요\n제14조(계약 전 알릴 의무)\n계약 전 알릴 의무는 무엇인가요\n제15조(계약 전 알릴 의무 위반의 효과)\n제25조(제1회 보험료 및 회사의\n보장이 언제 시작되나요\n보장개시)\n청약을 철회하고 싶어요 제18조(청약의 철회)\n계약을 취소 할...
1,genai-demo-index-v1,29.6042,일시납종신보험_20240101~,4. 상황별 목차 안내\n(1) 해약환급금 보증형\n계약의 계약에서 쓰이는\n제2조(용어의정의)\n체결 용어를 알고 싶어요\n제14조(계약 전 알릴 의무)\n계약 전 알릴 의무는 무엇인가요\n제15조(계약 전 알릴 의무 위반의 효과)\n보장이 언제 시작되나요 제25조(제1회 보험료 및 회사의 보장개시)\n청약을 철회하고 싶어요 제18조(청약의 철회)...
2,genai-demo-index-v1,9.1513,일시납종신보험_20240101~,수 있습니다.\n제 15 조 계약 전 알릴 의무 위반의 효과\n① 회사는 계약자 또는 피보험자가 제14조(계약 전 알릴 의무)에도 불구하고 고의 또는 중대한 과실로\n중요한 사항에 대하여 사실과 다르게 알린 경우에는 회사가 별도로 정하는 방법에 따라 계약을 해지하거\n나 보장을 제한할 수 있습니다. 그러나 다음 중 한 가지에 해당되는 때에는 계약을 ...
3,genai-demo-index-v1,8.8277,일시납종신보험_20240101~,"【‘계약자의 책임’의 예시】\n제7조(보험금 지급사유의 발생통지), 제8조(보험금의 청구), 제11조(주소변경통지), 제25조(제1회 보험\n료 및 회사의 보장개시), 제26조(제2회 이후 보험료의 납입) 등\n【연대(連帶)】\n2인 이상의 계약자가 각자 채무의 전부를 이행할 책임을 지는 것을 말하며(지분만큼 분할하여 책임\n을 지는 것과는 다름),..."
4,genai-demo-index-v1,8.3660,일시납종신보험_20240101~,"약관 목차\n보험약관 가이드북 7\nⅠ. 약관 이용 안내 8\nⅡ. 약관 요약서 15\nⅢ. 보험계약 일반사항 20\n한화생명 The스마트한 일시납종신보험 무배당[보증비용부과형](해약환급금 보증)  주계약 약관 [보장형  25\n계약, 적립형 계약, 스마트전환형 계약] \n제 1 관 목적 및 용어의 정의 제 1 조  목적 25\n제 2 조  용어의 ..."


# 6. 오픈 서치에 "유사 서치" 검색
- query 를 제공해서 실제로 유사한 내용이 검색이 되는지를 확인 합니다.

In [38]:
def query_semantic(vector, filter=[], k=5):
    QUERY_TEMPLATE = {
        "size": k,
        "query": {
            "bool": {
                "must": [
                    {
                        "knn": {
                            "vector_field": {
                                "vector": vector,
                                "k": k 
                            }
                        }
                    }
                ],
                "filter": filter
            }
        }
    }
    if len(filter) > 0:
        QUERY_TEMPLATE["query"]["bool"]["filter"].extend(filter)
    return QUERY_TEMPLATE

In [39]:
query = "계약 전 의무 사항은 무엇인가요"

response = os_client.search(
    body=query_semantic(llm_emb.embed_query(query)),
    index=index_name
)
result_to_dataframe(response)

,index_name,score,type,text
0,genai-demo-index-v1,0.0054,종신보험_20230901~,"게 작성한 때에는 계약을 해지하거나 보장을 제한할 수 있습니다)\n5. 보험설계사 등이 계약자 또는 피보험자에게 고지할 기회를 주지 않았거나 계약자 또는 피보험자가\n사실대로 고지하는 것을 방해한 경우, 계약자 또는 피보험자에게 사실대로 고지하지 않게 하였거나\n부실한 고지를 권유했을 때\n다만, 보험설계사 등의 행위가 없었다 하더라도 계약자 또는 ..."
1,genai-demo-index-v1,0.0054,종신보험_20230916~,"게 작성한 때에는 계약을 해지하거나 보장을 제한할 수 있습니다)\n5. 보험설계사 등이 계약자 또는 피보험자에게 고지할 기회를 주지 않았거나 계약자 또는 피보험자가\n사실대로 고지하는 것을 방해한 경우, 계약자 또는 피보험자에게 사실대로 고지하지 않게 하였거나\n부실한 고지를 권유했을 때\n다만, 보험설계사 등의 행위가 없었다 하더라도 계약자 또는 ..."
2,genai-demo-index-v1,0.0054,종신보험_20240101~,"게 작성한 때에는 계약을 해지하거나 보장을 제한할 수 있습니다)\n5. 보험설계사 등이 계약자 또는 피보험자에게 고지할 기회를 주지 않았거나 계약자 또는 피보험자가\n사실대로 고지하는 것을 방해한 경우, 계약자 또는 피보험자에게 사실대로 고지하지 않게 하였거나\n부실한 고지를 권유했을 때\n다만, 보험설계사 등의 행위가 없었다 하더라도 계약자 또는 ..."
3,genai-demo-index-v1,0.0053,종신보험_20240101~,"를 이용하여 체결하는 계약은 계약자의 동의를 얻어 청약내용, 보험료납입, 보험기간, 계약 전 알릴 의\n무, 약관의 중요한 내용 등 계약을 체결하는 데 필요한 사항을 질문 또는 설명하는 방법으로 약관의 중\n요한 내용을 설명할 수 있습니다. 이 경우 계약자의 답변과 확인내용을 음성 녹음함으로써 약관의 중요\n한 내용을 설명한 것으로 봅니다.\n【통신..."
4,genai-demo-index-v1,0.0053,종신보험_20230916~,"를 이용하여 체결하는 계약은 계약자의 동의를 얻어 청약내용, 보험료납입, 보험기간, 계약 전 알릴 의\n무, 약관의 중요한 내용 등 계약을 체결하는 데 필요한 사항을 질문 또는 설명하는 방법으로 약관의 중\n요한 내용을 설명할 수 있습니다. 이 경우 계약자의 답변과 확인내용을 음성 녹음함으로써 약관의 중요\n한 내용을 설명한 것으로 봅니다.\n【통신..."


- langchain의 similarity_search_with_score API를 활용하는 방법
    - [API: similarity_search_with_score](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html#langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.similarity_search)


In [40]:
from langchain.chains.question_answering import load_qa_chain

In [41]:
# search_types = ["approximate_search", "script_scoring", "painless_scripting"]
# space_types = ["l2", "l1", "linf", "cosinesimil", "innerproduct", "hammingbit"]
results = vector_db.similarity_search_with_score(
    query=query,
    k=5,
    search_type="approximate_search",
    space_type="l2",
    boolean_filter={
        "bool": {
            "filter": []
        }
    }
)

In [42]:
results[:3]

[(Document(page_content='게 작성한 때에는 계약을 해지하거나 보장을 제한할 수 있습니다)\n5. 보험설계사 등이 계약자 또는 피보험자에게 고지할 기회를 주지 않았거나 계약자 또는 피보험자가\n사실대로 고지하는 것을 방해한 경우, 계약자 또는 피보험자에게 사실대로 고지하지 않게 하였거나\n부실한 고지를 권유했을 때\n다만, 보험설계사 등의 행위가 없었다 하더라도 계약자 또는 피보험자가 사실대로 고지하지 않거나\n부실한 고지를 했다고 인정되는 경우에는 계약을 해지하거나 보장을 제한할 수 있습니다.\n② 회사는 제1항에 따라 계약을 해지하거나 보장을 제한할 경우에는 계약 전 알릴 의무 위반사실(계약\n해지 등의 원인이 되는 위반사실을 구체적으로 명시)뿐만 아니라 계약 전 알릴 의무 사항이 중요한 사\n항에 해당되는 사유 및 계약의 처리결과를 “반대증거가 있는 경우 이의를 제기할 수 있습니다”라는 문\n구와 함께 계약자에게 서면 또는 전자문서 등으로 알려 드립니다. 회사가 전자문서로 안내하고자 할 경\n우에는 계약자에게 서면 또는 「전자서명법」 제2조 제2호에 따른 전자서명으로 동의를 얻어 수신확인을\n조건으로 전자문서를 송신하여야 합니다. 계약자의 전자문서 수신이 확인되기 전까지는 그 전자문서는\n송신되지 않은 것으로 봅니다. 회사는 전자문서가 수신되지 않은 것을 확인한 경우에는 서면(등기우편\n등)으로 다시 알려드립니다.\n③ 제1항에 따라 계약을 해지하였을 때에는 제33조(해약환급금) 제1항 및 제2항에 따른 해약환급금을\n드리며, 보장을 제한하였을 때에는 보험료, 보험가입금액 등이 조정될 수 있습니다.\n④ 제14조(계약 전 알릴 의무)의 계약 전 알릴 의무를 위반한 사실이 보험금 지급사유 발생에 영향을\n미쳤음을 회사가 증명하지 못한 경우에는 제1항에도 불구하고 계약의 해지 또는 보장을 제한하기 이전\n까지 발생한 해당 보험금을 지급합니다.\n⑤ 회사는 다른 보험가입내역에 대한 계약 전 알릴 의무 위반을 이유로 계약을 해지하거나 보험금 지

# 7.사용자 정의 가능한 옵션
이제 벡터 저장소가 준비되었으므로 질문을 시작할 수 있습니다.

Vector Store를 둘러싸서 LLM 입력을 받는 LangChain에서 제공하는 래퍼를 사용할 수 있습니다.
이 래퍼는 뒤에서 다음 단계를 수행합니다.
- 질문을 입력합니다.
- 질문 임베딩 생성
- 관련 문서 가져오기
- 프롬프트에 문서와 질문을 채워 넣습니다.
- 프롬프트로 모델을 호출하고 사람이 읽을 수 있는 방식으로 답변을 생성합니다.

위 시나리오에서는 질문에 대한 상황 인식 답변을 빠르고 쉽게 얻을 수 있는 방법을 탐색했습니다. 이제 문서를 가져오는 방법을 사용자 정의할 수 있는 [RetrievalQA](https://python.langchain.com/en/latest/modules/chains/index_examples/Vector_db_qa.html)의 도움으로 더 사용자 정의 가능한 옵션을 살펴보겠습니다. `chain_type` 매개변수를 사용하여 프롬프트에 추가해야 합니다. 또한 검색해야 하는 관련 문서 수를 제어하려면 아래 셀에서 'k' 매개변수를 변경하여 다른 출력을 확인하세요. 많은 시나리오에서 LLM이 답변을 생성하는 데 사용한 소스 문서가 무엇인지 알고 싶을 수 있습니다. LLM 프롬프트의 컨텍스트에 추가된 문서를 반환하는 `return_source_documents`를 사용하여 출력에서 ​​해당 문서를 가져올 수 있습니다. 'RetrievalQA'를 사용하면 모델에 특정한 사용자 정의 [프롬프트 템플릿](https://python.langchain.com/en/latest/modules/prompts/prompt_templates/getting_started.html)을 제공할 수도 있습니다.

참고: 이 예에서는 Amazon Bedrock에서 LLM으로 Anthropic Claude를 사용하고 있습니다. 이 특정 모델은 입력이 'Human:' 아래에 제공되고 모델이 'Assistant:' 다음에 출력을 생성하도록 요청되는 경우 가장 잘 수행됩니다. 아래 셀에는 LLM이 기본 상태를 유지하고 컨텍스트 외부에서 응답하지 않도록 프롬프트를 제어하는 ​​방법의 예가 나와 있습니다.

### [[REF] Using langchain for Question Answering on Own Data](https://medium.com/@onkarmishra/using-langchain-for-question-answering-on-own-data-3af0a82789ed)

In [43]:
from langchain.schema import BaseRetriever
from typing import Any, Dict, List, Optional, List, Tuple
from langchain.callbacks.manager import CallbackManagerForRetrieverRun

# lexical(keyword) search based (using Amazon OpenSearch)
class OpenSearchLexicalSearchRetriever(BaseRetriever):
    os_client: Any
    index_name: str
    k = 3
    filter = []

    def normalize_search_results(self, search_results):
        hits = (search_results["hits"]["hits"])
        max_score = float(search_results["hits"]["max_score"])
        for hit in hits:
            hit["_score"] = float(hit["_score"]) / max_score
        search_results["hits"]["max_score"] = hits[0]["_score"]
        search_results["hits"]["hits"] = hits
        return search_results

    def update_search_params(self, **kwargs):
        self.k = kwargs.get("k", 3)
        self.filter = kwargs.get("filter", [])
        self.index_name = kwargs.get("index_name", self.index_name)

    def _reset_search_params(self, ):
        self.k = 3
        self.filter = []
        
    def query_lexical(self, query, filter=[], k=5):
        QUERY_TEMPLATE = {
            "size": k,
            "query": {
                "bool": {
                    "must": [
                        {
                            "match": {
                                "text": {
                                    "query": query,
                                    "operator":  "or"
                                }
                            }
                        }
                    ],
                    "filter": filter
                }
            }
        }
        
        if len(filter) > 0:
            QUERY_TEMPLATE["query"]["bool"]["filter"].extend(filter)
            
        return QUERY_TEMPLATE
    

    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun) -> List[Document]:
        
        query = self.query_lexical(
            query=query,
            filter=self.filter,
            k=self.k
        )

        # print ("lexical search query: ")
        # print(query)
        
        search_results = self.os_client.search(
            body=query,
            index=self.index_name
        )

        results = []
        if search_results["hits"]["hits"]:
            search_results = self.normalize_search_results(search_results)
            for res in search_results["hits"]["hits"]:

                metadata = res["_source"]["metadata"]
                metadata["id"] = res["_id"]

                doc = Document(
                    page_content=res["_source"]["text"],
                    metadata=metadata
                )
                results.append((doc))

        self._reset_search_params()

        return results[:self.k]


In [44]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from utils.rag import run_RetrievalQA, show_context_used

In [45]:
# prompt_template = """
# \n\nHuman: 다음 문맥의 Information을 사용하여 고객 서비스 센터 직원처럼, 마지막 질문에 대한 목차 형식으로 답변을 제공하세요. 응답을 모르면 모른다고 말하고 응답을 만들려고 하지 마세요.

# {context}

# Question: {question}
# \n\nAssistant:"""

prompt_template = """
\n\nHuman: Use the following pieces of context to provide a concise answer to the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}

\n\nAssistant:"""


PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

In [46]:
chain = load_qa_chain(
    llm=llm_text,
    chain_type="stuff",
    prompt=PROMPT,
    verbose=True
)

In [47]:
boolean_filter = []
boolean_filter = [
    {"term": {"metadata.source": "한화생명 The스마트한 일시납종신보험_20240101~.pdf"}},
    {"term": {"metadata.type": "일시납종신보험_20240101~"}},
]

In [48]:
opensearch_lexical_retriever = OpenSearchLexicalSearchRetriever(
    os_client=os_client,
    index_name=index_name,
    k=3,
    filter=boolean_filter
)

In [49]:
answer = chain.run(
    input_documents=opensearch_lexical_retriever.get_relevant_documents(query),
    question=query
)

self.input_keys : ['input_documents', 'question']


> Entering new StuffDocumentsChain chain...
self.input_keys : ['context', 'question']


> Entering new LLMChain chain...
Prompt after formatting:



Human: Use the following pieces of context to provide a concise answer to the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

(2) 해약환급금 미보증형
계약의 계약에서 쓰이는
제2조(용어의 정의)
체결 용어를 알고 싶어요
제14조(계약 전 알릴 의무)
계약 전 알릴 의무는 무엇인가요
제15조(계약 전 알릴 의무 위반의 효과)
제25조(제1회 보험료 및 회사의
보장이 언제 시작되나요
보장개시)
청약을 철회하고 싶어요 제18조(청약의 철회)
계약을 취소 할 수 있나요 제19조(약관교부 및 설명의무 등)
계약이 무효가 될 수 있나요 제20조(계약의 무효)
보험료의 보험료 납입면제
제5조(보험금 지급에 관한 세부규정)
납입 사유를 알고 싶어요
보험료 납입최고(독촉)이 제28조(보험료의 납입이 연체되는 경우
무엇인가요 납입최고(독촉)와 계약의 해지)
제29조(보험료의 납입연체로 인하여
해지된 계약을 살리고 싶어요
해지된 계약의 부활(효력회복))
보험금의 제4조(보험금의 지급사유),
보험금을 받을 수 있는지 궁금해요
지급 제6조(보험금을 지급하지 않는 사유)
제8조(보험금의 청구),
보험금은 언제 지급되나요
제9조(보험금의 지급절차)
11 / 249

4. 상황별 목차 안내
(1) 해약환급금 보증형
계약의 계약에서 쓰이는
제2조(용어의정의)
체결 용어를 알고 싶어요
제14조(계약 전 알릴 의무)
계약 전

In [50]:
opensearch_semantic_retriever = vector_db.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k": 5,
        "boolean_filter": boolean_filter
    }
)

In [51]:
answer = chain.run(
    input_documents=opensearch_semantic_retriever.get_relevant_documents(query),
    question=query
)

self.input_keys : ['input_documents', 'question']


> Entering new StuffDocumentsChain chain...
self.input_keys : ['context', 'question']


> Entering new LLMChain chain...
Prompt after formatting:



Human: Use the following pieces of context to provide a concise answer to the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

【‘계약자의 책임’의 예시】
제7조(보험금 지급사유의 발생통지), 제8조(보험금의 청구), 제11조(주소변경통지), 제25조(제1회 보험
료 및 회사의 보장개시), 제26조(제2회 이후 보험료의 납입) 등
【연대(連帶)】
2인 이상의 계약자가 각자 채무의 전부를 이행할 책임을 지는 것을 말하며(지분만큼 분할하여 책임
을 지는 것과는 다름), 계약자 중 1인이 책임을 이행하는 경우 나머지 계약자는 책임을 면하게 됩니
다.
제 3 관 계약자의 계약 전 알릴 의무 등
제 14 조 계약 전 알릴 의무
계약자 또는 피보험자는 청약할 때(진단계약의 경우에는 건강진단할 때를 말합니다) 청약서에서 질문한
사항에 대하여 알고 있는 사실을 반드시 사실대로 알려야(이하 ‘계약 전 알릴 의무’라 하며, 상법상 ‘고지
의무’와 같습니다)합니다. 다만, 진단계약에서 의료법 제3조(의료기관)의 규정에 따른 종합병원과 병원에
서 직장 또는 개인이 실시한 건강진단서 사본 등 건강상태를 판단할 수 있는 자료로 건강진단을 대신할
수 있습니다.
제 15 조 계약 전 알릴 의무 위반의 효과
① 회사는 계약자 또는 피보험자가 제14조(계약 전 알릴 의무)에도 불구하고 고의 또는 중대한 과실로
중요한 사항에 대

# 8. OpenSearch Hybrid 검색

OpenSearch Hybrid 는 아래와 같은 방식으로 작동합니다.
- (1) "Vector 서치" 하여 스코어를 얻은 후에 표준화를 하여 스코어를 구함. 
    - 전체 결과에서 가장 높은 스코어는 표준화 과정을 통하여 스코어가 1.0 이 됨.
- (2) Keyword 서치도 동일하게 함.
- (3) Reciprocal Rank Fusion (RRF) 기반 Re-rank
    - Paper: https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf
    - Desc: https://medium.com/@sowmiyajaganathan/hybrid-search-with-re-ranking-ff120c8a426d
    - **RRF의 경우 score가 아닌 ranking 정보를 활용, 때문에 score normalization이 필요 없음**

RRF는 langchain에서 "Ensemble Retriever" 이름으로 api를 제공합니다. 
- https://python.langchain.com/docs/modules/data_connection/retrievers/ensemble


### Ensemble retriever 정의
- https://python.langchain.com/docs/modules/data_connection/retrievers/ensemble
- RRF 방식만 지원
- Rank constant (param "c")
    - This value determines how much influence documents in individual result sets per query have over the final ranked result set. A higher value indicates that lower ranked documents have more influence. This value must be greater than or equal to 1. Defaults to 60.
    - 숫자 높을 수록 낮은 랭크의 문서가 더 중요시 된다

In [52]:
from langchain.retrievers import EnsembleRetriever

In [53]:
ensemble_retriever = EnsembleRetriever(
    retrievers=[opensearch_lexical_retriever, opensearch_semantic_retriever],
    weights=[0.5, 0.5],
    c=100,
    k=5
)

In [ ]:
%%time
answer = chain.run(
    input_documents=ensemble_retriever.get_relevant_documents(query),
    question=query
)

print("##############################")
print("query: \n", query)
print("answer: \n", answer)

self.input_keys : ['input_documents', 'question']


> Entering new StuffDocumentsChain chain...
self.input_keys : ['context', 'question']


> Entering new LLMChain chain...
Prompt after formatting:



Human: Use the following pieces of context to provide a concise answer to the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

4. 상황별 목차 안내
계약의 계약에서 쓰이는
제2조(용어의 정의)
체결 용어를 알고 싶어요
제14조(계약 전 알릴 의무)
계약 전 알릴 의무는 무엇인가요
제15조(계약 전 알릴 의무 위반의 효과)
보장이 언제 시작되나요 제24조(제1회 보험료 및 회사의 보장개시)
청약을 철회하고 싶어요 제18조(청약의 철회)
계약을 취소 할 수 있나요 제19조(약관교부 및 설명의무 등)
계약이 무효가 될 수 있나요 제20조(계약의 무효)
보험료의 보험료 납입면제
제5조(보험금 지급에 관한 세부규정)
납입 사유를 알고 싶어요
보험료 납입최고(독촉)이 제27조(보험료의 납입이 연체되는 경우
무엇인가요 납입최고(독촉)와 계약의 해지)
제28조(보험료의 납입연체로 인하여
해지된 계약을 살리고 싶어요
해지된 계약의 부활(효력회복))
보험금의 제4조(보험금의 지급사유),
보험금을 받을 수 있는지 궁금해요
지급 제6조(보험금을 지급하지 않는 사유)
제8조(보험금의 청구),
보험금은 언제 지급되나요
제9조(보험금의 지급절차)
8 / 165

【‘계약자의 책임’의 예시】
제7조(보험금 지급사유의 발생통지), 제8조(보험금의 청구), 제11조(주소변경통지), 제25조(제1회 보험
료 및 회사의 보장개시